# Main dataset + climate merge
**CS109A Final Project**<br>
**Predicting Food Inspection Outcomes in Chicago**<br>
Calvin J Chiew, Angelo Kastroulis, Tim Hagmann<br>

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## 1: Prepare inspection dataset

In [2]:
# load main dataset 
df_inspection = pd.read_csv('inspections_clean.csv')

# drop first unnamed column
df_inspection.drop(df_inspection.columns[[0]], axis=1, inplace=True)

# view first few rows
df_inspection.head()

,License #,risk_description,Zip,inspection_date_string,Y_description,Latitude,Longitude,Y,Y_fail,reinspection?,...,license?,risk,inspection_type,inspection_dt,prev_fail,cumulative_failures,ever_failed,cumulative_inspections,proportion_past_failures,days_since_last_inspection
0,2495906.0,Risk 3 (Low),60632.0,11/03/2016,Pass,41.821458,-87.704350,1.0,0.0,0,...,1,3,0.0,2016-11-03,0,0,0,1,0.0,NaN
1,2093906.0,Risk 2 (Medium),60615.0,11/03/2016,Pass,41.795175,-87.596601,1.0,0.0,0,...,0,2,1.0,2016-11-03,0,0,0,8,0.0,373.0
2,2495782.0,Risk 3 (Low),60628.0,11/03/2016,Pass,41.714126,-87.642853,1.0,0.0,0,...,1,3,0.0,2016-11-03,0,0,0,1,0.0,NaN
3,2476569.0,Risk 1 (High),60621.0,11/02/2016,Pass,41.779856,-87.645142,1.0,0.0,1,...,1,1,2.0,2016-11-02,1,1,1,2,0.5,8.0
4,2476568.0,Risk 1 (High),60621.0,11/02/2016,Pass,41.779856,-87.645142,1.0,0.0,1,...,1,1,2.0,2016-11-02,1,1,1,2,0.5,8.0


In [3]:
# generate table to assess extent of missing data
def missing_values_table(df): 
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0: 'Missing Values', 1: '% Missing Values'})
    return mis_val_table_ren_columns

missing_values_table(df_inspection)

,Missing Values,% Missing Values
License #,13,0.011000
risk_description,19,0.016077
Zip,84,0.071079
inspection_date_string,0,0.000000
Y_description,0,0.000000
Latitude,453,0.383317
Longitude,453,0.383317
Y,0,0.000000
Y_fail,0,0.000000
reinspection?,0,0.000000


## 2: Prepare climate dataset

From data exploration, we noted that high temperature appears to be associated with failed outcome. Here, we will extract daily maximum (TMAX) and minimum temperature (TMIN) recorded at CHICAGO NORTHERLY ISLAND station, which represents the most comprehensive record from years 2010 to present and is located closest to the city center.

In [5]:
# load climate dataset
df_climate = pd.read_csv('data/climate835468.csv')

# format datetime string
df_climate['DATE'] =  pd.to_datetime(df_climate['DATE'], format='%Y%m%d')

# extract rows for Northerly Island station, drop missing data
df_northerlyisland = df_climate[(df_climate['STATION_NAME'] == 'CHICAGO NORTHERLY ISLAND IL US') & (df_climate['TMAX'] != -9999) & (df_climate['TMIN'] != -9999)]

# drop all columns except date and temperature
df_temperature = df_northerlyisland[['DATE', 'TMAX', 'TMIN']]

# view first few rows
print 'Shape:', df_temperature.shape
df_temperature.head()

Shape: (2461, 3)


,DATE,TMAX,TMIN
17146,2010-01-01,21,10
17147,2010-01-02,16,7
17148,2010-01-03,24,6
17149,2010-01-04,21,13
17150,2010-01-05,27,19


Here, we will generate 2 more variables as defined below: <br>
- TMAX_3: three-day average high temperature, where TMAX_3 = mean of all TMAX of D, D-1, D-2 days
- TMAX_5: five-day average high temperature, where TMAX_5 = mean of all TMAX of D, D-1, D-2, D-3, D-4 days

In [6]:
# calculate rolling means
three_day_rolling = df_temperature['TMAX'].rolling(window=3, center=False).mean()
five_day_rolling = df_temperature['TMAX'].rolling(window=5, center=False).mean()

# convert to dataframes and concatenate to temperature dataset
df_three_day_rolling = pd.DataFrame(three_day_rolling).rename(columns={'TMAX' : 'TMAX_3'})
df_five_day_rolling = pd.DataFrame(five_day_rolling).rename(columns={'TMAX' : 'TMAX_5'})

# concatenate into one dataframe
df_temperature = pd.concat([df_temperature, df_three_day_rolling, df_five_day_rolling], axis=1)

# view first few rows
df_temperature.head(10)

,DATE,TMAX,TMIN,TMAX_3,TMAX_5
17146,2010-01-01,21,10,NaN,NaN
17147,2010-01-02,16,7,NaN,NaN
17148,2010-01-03,24,6,20.333333,NaN
17149,2010-01-04,21,13,20.333333,NaN
17150,2010-01-05,27,19,24.000000,21.8
17151,2010-01-06,25,15,24.333333,22.6
17152,2010-01-07,25,18,25.666667,24.4
17153,2010-01-08,31,20,27.000000,25.8
17154,2010-01-09,26,15,27.333333,26.8
17155,2010-01-10,21,6,26.000000,25.6


## 3: Merge datasets

The inspection and temperature datasets are merged on DATE and written out into new .csv file.

In [7]:
# format datetime string in inspection dataset
df_inspection["inspection_dt"] = pd.to_datetime(df_inspection["inspection_date_string"])

# rename date column in temperature dataset
df_temperature = df_temperature.rename(columns={'DATE' : 'inspection_dt'})
                                       
# left join
df_merged = pd.merge(df_inspection, df_temperature, how='left', on='inspection_dt')

# write out csv
df_merged.to_csv('inspection_merged_climate.csv')